In [1]:
import struct

In [2]:
f = open("seyoun_skeleton_neuroglancer.dms", "rb")
data = f.read()
num_verts = struct.unpack('<Q', data[0:8])[0]
num_edges = struct.unpack('<Q', data[8:16])[0]
bytes_per_vert = 12 + 4 # 12 from 3 verts and 4 from group ID
bytes_per_edge = 16
verts = []
edges = []
idx = 16
while idx <= num_verts * bytes_per_vert:
    vert = list(struct.unpack('<fff', data[idx:idx+12]))
    int_end = struct.unpack('<I', data[idx+12:idx+16])[0]
    vert = vert + [int_end]
#     print(vert)
    verts.append(list(vert))
    idx = idx + 16
while idx <= num_verts * bytes_per_vert + num_edges * bytes_per_edge:
    edge = struct.unpack('<QQ', data[idx:idx+16])
#     print(edge)
    edges.append(list(edge))
    idx = idx + 16
    
#assert(idx == len(data))
f.close()

In [3]:
print(len(verts), len(edges))

417498 417445


In [4]:
print(verts[42])

[1340.0, 2192.0, 275.0, 1]


In [5]:
x = [0, 14757]
y = [0, 20982]
z = [0, 1272]
voxel = [585, 585, 5000]

In [6]:
import numpy as np

In [7]:
print(verts[42:50])
int_verts = [[int(v) for v in vert] for vert in verts]
print(' ')
print(int_verts[42:50])

[[1340.0, 2192.0, 275.0, 1], [1342.0, 2190.0, 275.0, 1], [1342.0, 2188.5, 275.0, 1], [1341.0, 2189.5, 275.0, 1], [1342.0, 2190.5, 275.0, 1], [1340.5, 2190.0, 275.0, 1], [1342.0, 2191.0, 275.0, 1], [1340.0, 2193.0, 275.0, 1]]
 
[[1340, 2192, 275, 1], [1342, 2190, 275, 1], [1342, 2188, 275, 1], [1341, 2189, 275, 1], [1342, 2190, 275, 1], [1340, 2190, 275, 1], [1342, 2191, 275, 1], [1340, 2193, 275, 1]]


In [8]:
def vert2int(vert):
    ints = [int(v) for v in vert]
    return ints

In [9]:
collection = 'ailey-dev'
experiment = 's3617'
channel = 'channel1'
x_rng = [3000, 12000]
y_rng = [6500, 14000]
z_rng = [270, 540]

In [10]:
total_volume = np.zeros((x_rng[1]-x_rng[0],y_rng[1]-y_rng[0],z_rng[1])) #x,y,z
total_volume.shape

(9000, 7500, 540)

In [11]:
from tqdm import tqdm

In [12]:
print([i for i in range(-1,2)])

[-1, 0, 1]


In [13]:
def combinations(r):
    return [(i,j) for i in range(-1*r,r+1) for j in range(-1*r,r+1)]
print(len(combinations(1)))
print(combinations(1))

9
[(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 0), (0, 1), (1, -1), (1, 0), (1, 1)]


In [14]:
for edge in tqdm(edges):
    v1, v2 = edge
    vert1 = vert2int(verts[v1])
    vert2 = vert2int(verts[v2])
    #diffs = np.subtract(vert1,vert2)
    #if np.any(np.abs(diffs)>3):
    #print(vert1, vert2)
    color = vert1[-1]
    interp = [[int(c*vert1[0] + (1-c)*vert2[0]), int(c*vert1[1] + (1-c)*vert2[1]), int(c*vert1[2] + (1-c)*vert2[2])] for c in np.linspace(0,1,100)]
    unique_data = [list(x) for x in set(tuple(x) for x in interp)]
    #print('interp:',unique_data)
    #print('color:', color)
    for element in unique_data:
        for ind in combinations(5):
            total_volume[element[0]+ind[0]-3000, element[1]+ind[1]-6500, element[2]] = color

100%|██████████| 417445/417445 [04:28<00:00, 1556.28it/s]


In [15]:
import matplotlib.pyplot as plt
import tifffile as tif
#plt.imshow(total_volume[:,:,0]) #Needs to be in row,col order
#plt.show()
#tif.imsave('total_volume.tif', total_volume[0])

In [16]:
def block_compute(x_start, x_stop,
                  y_start, y_stop,
                  z_start, z_stop,
                  origin=(0, 0, 0),
                  block_size=(512, 512, 16)):

    # x
    x_bounds = range(origin[0], x_stop + block_size[0], block_size[0])
    x_bounds = [x for x in x_bounds if (x > x_start and x < x_stop)]
    if len(x_bounds) is 0:
        x_slices = [(x_start, x_stop)]
    else:
        x_slices = []
        for start_x in x_bounds[:-1]:
            x_slices.append((start_x, start_x + block_size[0]))
        x_slices.append((x_start, x_bounds[0]))
        x_slices.append((x_bounds[-1], x_stop))

    # y
    y_bounds = range(origin[1], y_stop + block_size[1], block_size[1])
    y_bounds = [y for y in y_bounds if (y > y_start and y < y_stop)]
    if len(y_bounds) is 0:
        y_slices = [(y_start, y_stop)]
    else:
        y_slices = []
        for start_y in y_bounds[:-1]:
            y_slices.append((start_y, start_y + block_size[1]))
        y_slices.append((y_start, y_bounds[0]))
        y_slices.append((y_bounds[-1], y_stop))

    # z
    z_bounds = range(origin[2], z_stop + block_size[2], block_size[2])
    z_bounds = [z for z in z_bounds if (z > z_start and z < z_stop)]
    if len(z_bounds) is 0:
        z_slices = [(z_start, z_stop)]
    else:
        z_slices = []
        for start_z in z_bounds[:-1]:
            z_slices.append((start_z, start_z + block_size[2]))
        z_slices.append((z_start, z_bounds[0]))
        z_slices.append((z_bounds[-1], z_stop))

    chunks = []
    for x in x_slices:
        for y in y_slices:
            for z in z_slices:
                chunks.append((x, y, z))
    return chunks


In [17]:
regions = block_compute(0, total_volume.shape[0],
                        0, total_volume.shape[1],
                        0, total_volume.shape[2],
                        origin=(3000,6500,0),
                        block_size=(512, 512, 16))

In [18]:
from intern.remote.boss import BossRemote
from intern.resource.boss.resource import ChannelResource

In [19]:
rmt = BossRemote('../meshes/neurodata.cfg')

COLL_NAME = collection
EXP_NAME = experiment
CHAN_NAME = 'test_upload'

# Create a new channel that uses uint16 data.
chan_setup = ChannelResource(CHAN_NAME, COLL_NAME, EXP_NAME, 'annotation', datatype='uint64')
chan = rmt.get_project(chan_setup)

In [23]:
print(regions[1158])

((0, 3000), (0, 6500), (48, 64))


In [ ]:
for region in tqdm(regions[1157:]):#1156?
    x,y,z = region
    data_cut = np.array(total_volume[x[0]:x[1], y[0]:y[1], z[0]:z[1]], dtype='uint64')
    rmt.create_cutout(chan, 0, x, y, z, np.ascontiguousarray(data_cut))